In [ ]:
USE_WIDGETS = True

def importEssentialLibs(USE_WIDGETS):
    import numpy as np
    if USE_WIDGETS:
        %matplotlib widget
    else:
        %matplotlib inline
    import matplotlib as mpl
    mpl.rc('text', usetex = True)
    mpl.rc('font', family = 'serif', size = 18)
    import matplotlib.pyplot as plt
    import scipy.signal as signal
    
    return np, mpl, plt, signal

In [ ]:
import numpy as np
import numpy.fft as fft
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family  = 'serif', size = 18)
import matplotlib.pyplot as plt
import math
import scipy.signal as signal
import scipy.io.wavfile as wav
import scipy.io as sio
import IPython
from IPython.display import Markdown

1.1

In [ ]:
import scipy.io as sio
import math as math
matContents = sio.loadmat('dz2_signali/ecg_corrupted.mat')
ecg = matContents['ecg_corrupted'].squeeze()
fs=360
t=np.arange(len(ecg))/fs;
fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)
plt.plot(t,ecg)
plt.xlabel('t[s]')
plt.ylabel('ecg(t)')
plt.title('ecg corrupted')

1.2

In [ ]:
def baselineDriftFilter(fs, fa, fp, Aa, Ap):
    # Specifikacije digitalnog filtra
    Wa=2*np.pi*(fa/fs)
    Wp=2*np.pi*(fp/fs)
    Ts=1/fs
    # Specifikacije analognog prototipa
    wa = 2/Ts*math.tan(Wa/2)
    wp = 2/Ts*math.tan(Wp/2)
    
    waN=1
    wpN = wa/wp    # Normalizovani NF prototip
    k = wpN/waN 
    D = (10**(0.1*Aa)-1) / (10**(0.1*Ap)-1)
    N = math.ceil(np.arccosh(np.sqrt(D)) / (np.arccosh(1/k)))
    
    z, p, k = signal.cheb2ap(N, Aa)
    
    an = np.array(np.poly(p))
    bn= k*np.array(np.poly(z))
    #u slucaju da nema korenova moramo napraviti niz, makar bio i jedinicni jer lp2hp prima nizove
    if an.shape == ():
        an = np.array([an])
    if bn.shape == ():
        bn = np.array([bn])
        
    b, a = signal.lp2hp(bn, an, wa);
    
    [bd, ad] = signal.bilinear(b, a, fs)
    
    return  bd, ad
        

1.4

In [ ]:

np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)

fs = 360
fa = 0.4
fp = 1
Aa = 30
Ap = 0.5

bd, ad = baselineDriftFilter(fs, fa, fp, Aa, Ap)
#E=fft.fft(ecg,1000)
s = signal.lfilter(bd, ad, ecg)
t = np.arange(len(s))/fs
#S=fft.fft(s,1000)
fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.plot(t, s)
ax.set_ylabel('$ECG$')
ax.set_xlabel('$t$')
ax.set_title('Filtrirani EKG signal')


1.3

In [ ]:
def powerLineNoiseFilter(fs, fc, Aa, Ap):
    Ts=1/fs
    Fp1=(fc-2)/fs
    Fp2=(fc+2)/fs
    Fa1=(fc-0.5)/fs
    Fa2=(fc+0.5)/fs
    
    Wp1=2*np.pi*Fp1
    Wp2=2*np.pi*Fp2
    Wa1=2*np.pi*Fa1
    Wa2=2*np.pi*Fa2
    #pripremamo kruzne ucestanosti unapred za bilinearnu transformaciju
    wp1 = 2/Ts*math.tan(Wp1/2)
    wp2 = 2/Ts*math.tan(Wp2/2)
    wa1 = 2/Ts*math.tan(Wa1/2)
    wa2 = 2/Ts*math.tan(Wa2/2)
    
    wpN=1
    
    
    w0 = np.sqrt(wp1*wp2)
    B = (wp2 - wp1)*wpN
    waN = (B*wa1)/(w0**2 - wa1**2)
   
    k = wpN/waN 
    D = (10**(0.1*Aa)-1) / (10**(0.1*Ap)-1)
    N = math.ceil(np.arccosh(np.sqrt(D)) / (np.arccosh(1/k)))
    z, p, k = signal.cheb1ap(N, Ap)
    
    an = np.array(np.poly(p))
    bn = k*np.array(np.poly(z))
    if an.shape == ():
        an = np.array([an])
    if bn.shape == ():
        bn = np.array([bn])
        
    b, a = signal.lp2bs(bn, an, w0, B);
    
    [bd, ad] = signal.bilinear(b, a, fs)
   #probaj da vidis sta se dobije kad ne uradis tranformaciju ucestanosti po tangensnom prenosu  u koje se ucestanosti to pretvori 
    return bd,ad
    
    

1.5

In [ ]:
fs = 360
fc = 60
Aa = 40
Ap = 0.5

bd, ad = powerLineNoiseFilter(fs, fc, Aa, Ap)
filtered_2 = signal.lfilter(bd, ad, s)
t = np.arange(len(filtered_2))/fs
fig, ax = plt.subplots(1, 1, figsize = [10, 5])
plt.subplots_adjust(bottom=0.2, wspace = 0.3)

ax.plot(t, filtered_2)
ax.set_ylabel('$ECG_filtered$')
ax.set_xlabel('$t$')
ax.set_title('Filtrirani EKG signal')

1.6

In [ ]:
fig ,ax= plt.subplots(2,1,figsize = (10,5))
plt.subplots_adjust(wspace = 2, hspace = 1)
fs = 360
fa = 0.4
fp = 1
Aa = 30
Ap = 0.5

bd, ad = baselineDriftFilter(fs, fa, fp, Aa, Ap)
wd_4, Hd_4 = signal.freqz(bd, ad, worN = 10000)
fd_4=wd_4/(2*np.pi)*fs#
'''cilje je pretvoriti relativne kruzne u prave ucestanosti, jer tako proveravamo da li je na amplitudskoj karakteristici napravljan trazeni filtar'''
Hda_4 = abs(Hd_4)

ax[0].semilogx(fd_4, 20*np.log10(Hda_4), label='funkcija prenosa iz tacke 1.4')
#ax[0].set_xlim(0,5)

Wa = fa/fs*2*np.pi
Wp = fp/fs*2*np.pi


# Dizajn filtra
wpVector = np.array([Wp])/np.pi;
waVector = np.array([Wa])/np.pi;
bd, ad = signal.iirdesign(wpVector, waVector, Ap, Aa, analog=False, ftype='cheby2', output='ba', fs=None)

fd_irrd, Hd_irrd = signal.freqz(bd, ad, worN = 10000,fs=fs)
#fd_irrd=wd_irrd/(2*np.pi)*fs

Hda_irrd=abs(Hd_irrd)

ax[0].semilogx(fd_irrd, 20*np.log10(Hda_irrd), label='funkcija prenosa pomocu irrdesign funkcije')

ax[0].set_ylabel('$|H_4(e^{j\Omega})|, |H_{irr}(e^{j\Omega})|$')
ax[0].set_xlabel('$f$');
ax[0].set_title('Ampltiduske karakteristike VF filtara')
ax[0].legend(loc = 'best')





fs = 360
fc = 60
Aa = 40
Ap = 0.5

bd, ad = powerLineNoiseFilter(fs, fc, Aa, Ap)
fd_5, Hd_5 = signal.freqz(bd, ad, worN = 10000,fs=fs)
Hda_5 = abs(Hd_5)

ax[1].semilogx(fd_5, 20*np.log10(Hda_5), label='funkcija prenosa iz tacke 1.5')
#ax[0].set_xlim(0,5)

Fp1=(fc-2)/fs
Fp2=(fc+2)/fs
Fa1=(fc-0.5)/fs
Fa2=(fc+0.5)/fs
    
Wp1=2*np.pi*Fp1
Wp2=2*np.pi*Fp2
Wa1=2*np.pi*Fa1
Wa2=2*np.pi*Fa2


# Dizajniranje filtra pomocu ugradjene funkcije
wpVector = np.array([Wp1,Wp2])/np.pi;
waVector = np.array([Wa1,Wa2])/np.pi;
bd, ad = signal.iirdesign(wpVector, waVector, Ap, Aa, analog=False, ftype='cheby1', output='ba', fs=None)

wd_irrd_5, Hd_irrd_5 = signal.freqz(bd, ad, worN = 10000)
fd_irrd=wd_irrd_5/(2*np.pi)*fs

Hda_irrd_5=abs(Hd_irrd_5)

ax[1].semilogx(fd_irrd, 20*np.log10(Hda_irrd_5), label='funkcija prenosa promocu irrdesign funkcije')

ax[1].set_ylabel('$|H_5(e^{j\Omega})|, |H_{irr}(e^{j\Omega})|$')
ax[1].set_xlabel('$f$');
ax[1].set_title('Ampltiduske karakteristike NO filtara')
ax[1].legend(loc='best')



1.7

In [ ]:
from dosutils import zplane

fig = plt.figure(figsize = [19,5])
plt.subplots_adjust(wspace = 0.5,hspace=2)
fs = 360
fa = 0.4
fp = 1
Aa = 30
Ap = 0.5

bd, ad = baselineDriftFilter(fs, fa, fp, Aa, Ap)
wd_4, Hd_4 = signal.freqz(bd, ad, worN = 10000)
#preskaliranje digitalnih ucestanosti u realne jer korisnika sa strane ne zanimaju realne ucestanosti nego ga zanima da analogne signale sa ulaza digitalnom filtracijom na ulazu izgubi analogne komponente,dakle analogno ga zanima a ne 
fd_4=wd_4/(2*np.pi)*fs
ax1=fig.add_subplot(1,3,1)
Hda_4 = abs(Hd_4)
zplane(bd, ad, ax1)
ax1.set_title('Raspored nula i polova')

ax2=fig.add_subplot(1,3,2)
ax2.semilogx(fd_4, 20*np.log10(Hda_4), label='funkcija prenosa iz tacke 1.4')
ax2.set_title('Amplitudska karakteristika')
ax2.set_ylabel('$ |H(e^{j\Omega})|$')
ax2.set_xlabel('$f$ [Hz]');


ax3 = fig.add_subplot(1,3,3)
ax3.semilogx(fd_4, np.unwrap(np.angle(Hd_4)))# sluzi da se fazna karakteristika samo nadoveze da ne bi bilo skokova oko Pi tj. -Pi
ax3.set_title('Fazna karakteristika')
ax3.set_ylabel('$X(j\omega)$')
ax3.set_xlabel('$f$');
ax3.set_xlim([0.1, 100])

rXlim = 100
ax2.set_xlim([0, rXlim])
ax2.set_ylim([-55, 1])


ax2.add_patch(mpl.patches.Polygon([[0, -Aa ], [fa, -Aa], [fa, -Aa+2], [0, -Aa+2]], closed=True,fill=False, hatch='////', color = 'red'))

ax2.add_patch(mpl.patches.Polygon([[fp, -Ap-2 ], [rXlim, -Ap-2], [rXlim, -Ap], [fp, -Ap]], closed=True,fill=False, hatch='////', color = 'red'))


fig1 = plt.figure(figsize = [19,5])
plt.subplots_adjust(wspace = 0.5,hspace=2)

fs = 360
fc = 60
Aa = 40
Ap = 0.5

fp1=(fc-2)
fp2=(fc+2)
fa1=(fc-0.5)
fa2=(fc+0.5)

bd, ad = powerLineNoiseFilter(fs, fc, Aa, Ap)

wd_5, Hd_5 = signal.freqz(bd, ad, worN = 10000)
fd_5=wd_5/(2*np.pi)*fs
Hda_5 = abs(Hd_5)

ax1=fig1.add_subplot(1,3,1)
Hda_5 = abs(Hd_5)
zplane(bd, ad, ax1)
ax1.set_title('Raspored nula i polova')

ax2=fig1.add_subplot(1,3,2)
ax2.semilogx(fd_5, 20*np.log10(Hda_5), label='funkcija prenosa iz tacke 1.4')
ax2.set_title('Amplitudska karakteristika')
ax2.set_ylabel('$ |H(e^{j\Omega})|$')
ax2.set_xlabel('$f$ [Hz]');
ax2.set_xlim(50,70);

ax3 = fig1.add_subplot(1,3,3)
ax3.semilogx(fd_5, np.unwrap(np.angle(Hd_5)))
ax3.set_title('Fazna karakteristika')
ax3.set_ylabel('$X(j\omega)$')
ax3.set_xlabel('$f[Hz]$');
rXlim=70
ax3.set_xlim(50,rXlim);


ax2.add_patch(mpl.patches.Polygon([[0, -Ap-10 ], [fp1, -Ap-10], [fp1, -Ap], [0, -Ap]], closed=True,fill=False, hatch='////', color = 'red'))

ax2.add_patch(mpl.patches.Polygon([[fa1, -Aa+10 ], [fa2, -Aa+10], [fa2, -Aa], [fa1, -Aa]], closed=True,fill=False, hatch='////', color = 'red'))

ax2.add_patch(mpl.patches.Polygon([[fp2, -Ap-10 ], [rXlim, -Ap-10], [rXlim, -Ap], [fp2, -Ap]], closed=True,fill=False, hatch='////', color = 'red'))

2.1

In [ ]:
from scipy.io import wavfile
import IPython
import IPython.display as ipd
fs,x = wavfile.read('dz2_signali/sound_corrupted.wav')
fig,ax=plt.subplots(1,1,figsize=[10,5])
plt.subplots_adjust(bottom=0.25, hspace = 0.5)

IPython.display.display(IPython.display.Audio(x, rate = fs))

def forceAspect(ax, aspect=1):
    im = ax.get_images()
    extent =  im[0].get_extent()
    ax.set_aspect(abs((extent[1]-extent[0])/(extent[3]-extent[2]))/aspect)


t = np.arange(len(x))/fs

fMaxShow = fs/2


Nwin = 1024*2;
fMaxShow = fs//2
window = signal.blackman(Nwin)
NFFT = Nwin

f, t, Sxx = signal.spectrogram(x, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                                   scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs




ax.matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(ax, 3)

ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$f$ [Hz]');


Potrebno je primeniti tri nepropusnika opsega i time komponente suma koje su na 1510Hz,2710Hz i u siroj okolini oko 11000 potisnuti. 
Prvu komponentu cemo da potiskujemo u gabaritima: fa1=1450Hz,fa2=1580Hz,fp1=1400Hz,fp2=1650Hz. 
Drugu komponentu cemo da potiskujemo u gabaritima: fa1=2650Hz,fa2=2780Hz,fp1=2600Hz,fp2=2850Hz.
Trecu komponentu cemo da potiskujemo u gabaritima: fa1=6500Hz,fa2=15000Hz,fp1=6400Hz,fp2=15100Hz. 

2.2-2.4

In [ ]:
def OTPmethodFilter(fs,fp1,fa1,fa2,fp2,Aa,Ap): ##optimizaciona metoda
    
    Fp1=fp1/fs
    Fa1=fa1/fs
    Fa2=fa2/fs
    Fp2=fp2/fs
    
    Wp1=2*np.pi*(Fp1)
    Wa1=2*np.pi*(Fa1)
    Wa2=2*np.pi*(Fa2)
    Wp2=2*np.pi*(Fp2)
    
    dp = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    da = 10**(-0.05*Aa)
    
    D = (0.01201*np.log10(dp)**2 + 0.09664*np.log10(dp) - 0.51325)*np.log10(da) + (0.00203*np.log10(dp)**2 - 0.57054*np.log10(dp) - 0.44314)

    f = -16.9 + 14.6*(np.log10(dp) - np.log10(da))
    Bt = min([Wa1-Wp1,Wp2-Wa2])
    
    N = int(np.ceil(2*np.pi*D/Bt - f*Bt/(2*np.pi) + 1))
    F=[0,Fp1,Fa1,Fa2,Fp2,0.5]
    Hid=[1,0,1]
    W=[1,dp/da,1]
    
    numOFPoints = 1024
    filterFitsInSpecs = False
    first = True
    while not filterFitsInSpecs:
        h = signal.remez(N, F, Hid, W)
        w, H = signal.freqz(h, 1, worN =numOFPoints )
        Hdb = 20*np.log10(abs(H))
        if first:
            first=False
            Horiginal=Hdb
            #posto su gornje formule empirijske okvirne, treba proveriti ipak da li su gabariti zadovoljeni za dati red koje su formule dobile 
        step= np.pi/numOFPoints
        indexWp1 = int(np.ceil(Wp1/step))
        indexWa1 = int(np.floor(Wa1/step))
        indexWa2 = int(np.ceil(Wa2/step))
        indexWp2 = int(np.floor(Wp2/step))
        
        if np.all(Hdb[:indexWp1] > -Ap) and np.all(Hdb[:indexWp1] < Ap) and \
            np.all(Hdb[indexWa1 :indexWa2] < - Aa) and np.all(Hdb[indexWp2:] < Ap) and np.all(Hdb[indexWp2:] > -Ap) :
            filterFitsInSpecs = True 
        else:
            N += 1
            
        return N,h

In [ ]:
np, mpl, plt, signal = importEssentialLibs(USE_WIDGETS)
def OIOmethodFilter(fs,fp1,fa1,fa2,fp2,Aa,Ap):#metoda ogranicavanja impulsnog odziva 
    
    correctNUMpoints=True
    
    Wp1=2*np.pi*(fp1/fs)
    Wa1=2*np.pi*(fa1/fs)
    Wa2=2*np.pi*(fa2/fs)
    Wp2=2*np.pi*(fp2/fs)
    
    Bt = min(Wa1 - Wp1,Wp2-Wa2)
    
    # Delta
    delta_p = (10**(0.05*Ap)-1)/(10**(0.05*Ap)+1)
    delta_a = 10**(-0.05*Aa)
    
    delta = min([delta_p, delta_a])
    AaSpec = -20*np.log10(delta)
        
    # Beta faktor
    beta = 0;
    if AaSpec < 21:
        beta = 0
    elif AaSpec <= 50:
        beta = 0.5842*(AaSpec-21)**0.4 + 0.07886*(AaSpec-21)
    else:
        beta = 0.1102*(AaSpec-8.7)
    
    # Red filtra
    if AaSpec <= 21:
        D = 0.9222
    else:
        D = (AaSpec-7.95)/14.36
    M = int(np.ceil(2*np.pi*D/Bt + 1))
    N = M - 1
    Noriginal=N;
    
    Wc1 = (Wa1 + Wp1)/2
    Wc2 = (Wa2 + Wp2)/2
    
    correctNUMpoints=True
    firstAmplitute=True;
    while correctNUMpoints:
        w=signal.kaiser(N+1,beta)
        
        na=np.arange(-N/2,N/2+1,1)
        n=np.arange(N+1)
        hID=np.zeros(N+1)
        if N%2==0:
            na=np.delete(na,N//2) # hocemo da izbrisemo iz niza 0 zbog deljenja sa nulom u toj situaciji koja ce nastati zbog resenja integrala
            hID[n != N//2] = np.sin(np.pi*na)/np.pi/na + np.sin(Wc1*na)/np.pi/na - np.sin(Wc2*na)/np.pi/na
            hID[N//2] = 1 + Wc1/np.pi - Wc2/np.pi  # ovde smo umesto deljenja sa 0 ubacili rezultat limesa u nuli
        else :
            hID = np.sin(np.pi*na)/np.pi/na + np.sin(Wc1*na)/np.pi/na - np.sin(Wc2*na)/np.pi/na
        
        h=w*hID
       
        numPoints=1024*4
        W,H=signal.freqz(h,a=1,worN=numPoints)
        Habs=20*np.log10(abs(H))
        
        if firstAmplitute:
            firstAmplitute=False;
            HabsOrg=Habs
        
        
        step=np.pi/numPoints
        indexWp1=int(np.ceil(Wp1/step))
        indexWa1=int(np.floor(Wa1/step))
        indexWa2=int(np.ceil(Wa2/step))
        indexWp2=int(np.floor(Wp2/step))
        
        if np.all(Habs[:indexWp1]>-Ap) and np.all(Habs[:indexWp1]<Ap) and \
           np.all(Habs[indexWa1:indexWa2]<-Aa) and np.all(Habs[indexWp2:]>-Ap) and \
           np.all(Habs[indexWp2:]<Ap):
            
            
            correctNUMpoints=False
            
        else:
            N=N+1
        
    return N,h
        
        
            
        
        
    
    

In [ ]:
np, mpl, plt, signal= importEssentialLibs(USE_WIDGETS)

#filtriranje frekvencijske komponente oko 1510 Hz
N1, h = OIOmethodFilter(fs, 1400, 1450, 1580, 1650, 40, 1)
x1 = signal.lfilter(h,1,x)## zato sto je H(z)=sum(h[n]*z^(-n)) samo imenioc postavimo na 1 i bn je zapravo isto sto i h[n]

#filtriranje frekvencijske komponente oko 2750 Hz
N2, h = OIOmethodFilter(fs, 2600, 2650, 2780, 2850, 40, 1)
y1 = signal.lfilter(h,1,x1)

#filtriranje frekvencijske komponente oko 1550 Hz
N3, h = OIOmethodFilter(fs, 6400, 6500, 15000, 15100, 40, 1)
z1 = signal.lfilter(h,1,y1)


print("N1 OIOmethod=", N3)
print("N2 OIOmethod=", N2)
print("N3 OIOmethod=", N1)
    
IPython.display.display(IPython.display.Audio(z1, rate = fs))

In [ ]:
#filtriranje frekvencijske komponente oko 1510 Hz
N1, h = OTPmethodFilter(fs, 1400, 1450, 1580, 1650, 40, 1)
x2 = signal.lfilter(h,1,x)## zato sto je H(z)=sum(h[n]*z^(-n)) samo imenioc postavimo na 1 i bn je zapravo isto sto i h[n]

#filtriranje frekvencijske komponente oko 2750 Hz
N2, h = OTPmethodFilter(fs, 2600, 2650, 2780, 2850, 40, 1)
y2 = signal.lfilter(h,1,x2)

#filtriranje frekvencijske komponente oko 11000 Hz
N3, h = OTPmethodFilter(fs, 6400, 6500, 15000, 15100, 40, 0.5)
z2 = signal.lfilter(h,1,y2)


print("N1 OTPmethod=", N3)
print("N2 OTPmethod=", N2)
print("N3 OTPmethod=", N1)
    
IPython.display.display(IPython.display.Audio(z2, rate = fs))

In [ ]:
fig,ax=plt.subplots(2,1,figsize=[10,10])
plt.subplots_adjust(bottom=0.25, hspace = 0.5)
t = np.arange(len(z1))/fs

fMaxShow = fs/2


Nwin = 1024*2;
fMaxShow = fs//2
window = signal.hamming(Nwin)
NFFT = Nwin

f, t, Sxx = signal.spectrogram(z1, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                                   scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs




ax[0].matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(ax[0], 3)

ax[0].set_xlabel('$t$ [s]')
ax[0].set_ylabel('$f$ [Hz]');
ax[0].set_title('Metod ogranicavanja impulsnog odziva');


t = np.arange(len(z2))/fs

fMaxShow = fs/2


Nwin = 1024*2;
fMaxShow = fs//2
window = signal.hamming(Nwin)
NFFT = Nwin

f, t, Sxx = signal.spectrogram(z2, fs = fs, window=window, noverlap=Nwin//4, nfft=NFFT, return_onesided=True, 
                                   scaling='spectrum', mode='complex')
fMaxIndex = NFFT*fMaxShow//fs




ax[1].matshow(20*np.log10(abs(Sxx[:][:fMaxIndex])), extent=[min(t), max(t), min(f), fMaxShow], origin='lower', cmap = 'inferno')
forceAspect(ax[1], 3)

ax[1].set_xlabel('$t$ [s]')
ax[1].set_ylabel('$f$ [Hz]');
ax[1].set_title('Optimizacioni metod');

2.5

Crtanje amplitudskih karakteristika filtara. Prva tri su uradjeni metodom ogranicavanja impulsnog odziva , a ostali su dobijeni optimizacionom metodom.

In [ ]:
fig, ax = plt.subplots(3,1, figsize = [10, 20])
plt.subplots_adjust(bottom=0.15, wspace = 0.4)
Aa=40
Ap=1
numOFpoints=8192
#prvi
fp1=1400
fa1=1450
fa2=1580
fp2=1650
N1, h = OIOmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 1)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[0].plot(f,20*np.log10(abs(H)));
ax[0].set_title('NO filtar oko 1510Hz metodom ograničavanja impulsnog odziva')
ax[0].set_xlabel(r'$f$ [Hz]');
ax[0].set_ylabel(r'$H$[dB]');
ax[0].set_xlim([0, 2200]);
# First pass band lim
ax[0].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Stop band lim
ax[0].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
# Second pass band lim
ax[0].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
    
    
#drugi
fp1=2600
fa1=2650
fa2=2780
fp2=2850
N2, h = OIOmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 1)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[1].plot(f,20*np.log10(abs(H)));
ax[1].set_title('NO filtar oko 2710Hz metodom ograničavanja impulsnog odziva')
ax[1].set_xlabel(r'$f$ [Hz]');
ax[1].set_ylabel(r'$H$[dB]');
ax[1].set_xlim([0, 3500]);

ax[1].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[1].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[1].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))



#treci 
fp1=6400
fa1=6500
fa2=15000
fp2=15100
N3, h = OIOmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[2].plot(f,20*np.log10(abs(H)));
ax[2].set_title('NO filtar oko 11000Hz metodom ograničavanja impulsnog odziva')
ax[2].set_xlabel(r'$f$ [Hz]');
ax[2].set_ylabel(r'$H$[dB]');
ax[2].set_xlim([0, 20000]);
# First pass band lim
ax[2].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[2].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[2].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
  


In [ ]:
fig, ax = plt.subplots(3,1, figsize = [10, 20])
plt.subplots_adjust(bottom=0.15, wspace = 0.4)
Aa=40
Ap=1
numOFpoints=8192
#prvi
fp1=1400
fa1=1450
fa2=1580
fp2=1650
N1, h = OTPmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 1)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[0].plot(f,20*np.log10(abs(H)));
ax[0].set_title('NO filtar oko 1510Hz optimizacionom metodom')
ax[0].set_xlabel(r'$f$ [Hz]');
ax[0].set_ylabel(r'$H$[dB]');
ax[0].set_xlim([0, 2200]);

ax[0].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[0].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[0].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
    
    
#drugi
fp1=2600
fa1=2650
fa2=2780
fp2=2850
N2, h = OTPmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 1)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[1].plot(f,20*np.log10(abs(H)));
ax[1].set_title('NO filtar oko 2710Hz optimizacionom metodom')
ax[1].set_xlabel(r'$f$ [Hz]');
ax[1].set_ylabel(r'$H$[dB]');
ax[1].set_xlim([0, 3500]);

ax[1].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[1].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[1].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))



#treci 
fp1=6400
fa1=6500
fa2=15000
fp2=15100
N3, h = OTPmethodFilter(fs, fp1, fa1, fa2, fp2, 40, 0.5)
W, H = signal.freqz(h, 1 , worN = numOFpoints)
f=W*fs/(2*np.pi)
ax[2].plot(f,20*np.log10(abs(H)));
ax[2].set_title('NO filtar oko 11000Hz optimizacionom metodom')
ax[2].set_xlabel(r'$f$ Hz');
ax[2].set_ylabel(r'$H$[dB]');
ax[2].set_xlim([0, 20000]);

ax[2].add_patch(mpl.patches.Polygon([[0, -Ap-2 ], [fp1, -Ap-2], [fp1, -Ap], [0, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[2].add_patch(mpl.patches.Polygon([[fa1, -Aa ], [fa2, -Aa], [fa2, -Aa+2], [fa1, -Aa+2]], closed=True,
                                          fill=False, hatch='////', color = 'red'))

ax[2].add_patch(mpl.patches.Polygon([[fp2, -Ap-2 ], [fs, -Ap-2], [fs, -Ap], [fp2, -Ap]], closed=True,
                                          fill=False, hatch='////', color = 'red'))
  

Tacka 3


3.1

In [ ]:
def iirDirectTransposedAlternativeVersion(b, a, x):
    
    N=len(x)
    bk=b
    ak=np.zeros(len(b))
    ak[:len(a)]=a##sada imamo nizove ak i bk napravljeni tako da se dopuni ak nulama onoliko koliko je b niz veci od a 
    leftLineCurrent=np.zeros(len(b))##zbog for petlje smo stavili da currenta ima jedan vise
    leftLineNext=np.zeros(len(b)-1)
    rightLineCurrent=np.zeros(len(b))##zbog for petlje smo stavili da currenta ima jedan vise
    rightLineNext=np.zeros(len(b)-1)
    

    y=np.zeros(N)
    
    for n in range(N):
                
                leftLineCurrent[:len(b)-1]=leftLineNext
                rightLineCurrent[:len(bk)-1]=rightLineNext
                z=leftLineCurrent[0]+x[n]
                y[n]=bk[0]*z+rightLineCurrent[0]
                
                leftLineNext=leftLineCurrent[1:]-ak[1:]*z
                rightLineNext=rightLineCurrent[1:]+bk[1:]*z
               
    return y           
        
    
                
    
    

In [ ]:
def iirDirectTransposed(b, a, x):
    
    N=len(x)
    bk=b
    ak=np.zeros(len(b))
    ak[:len(a)]=a
    LineCurrent=np.zeros(len(b))##zbog for petlje smo stavili da currenta ima jedan vise
    LineNext=np.zeros(len(b)-1)
 
    y=np.zeros(N)
    
    for n in range(N):
                LineCurrent[:len(bk)-1]=LineNext
                y[n]=bk[0]*x[n]+LineCurrent[0]
                
                LineNext=bk[1:]*x[n]+LineCurrent[1:]-ak[1:]*y[n]
                #for i in range(1,len(ak)):
                 #   LineNext[i-1]=bk[i]*x[n]+LineCurrent[i]-ak[i]*y[n]
                    
    return y           
        

3.2

In [ ]:
from fxpmath import Fxp

def iirDirectTransposedFXP(b, a, x, outFxpFormat):
        N=len(x)
        bk=b
        ak=Fxp(np.zeros(len(b)),like=a)
        ak[:len(a)]=a##sada imamo nizove ak i bk napravljeni tako da se dopuni ak nulama onoliko koliko je b niz veci od a 
        M=len(b)
    
        if outFxpFormat == None:##ukoliko nije zadat format izlaznog broja mi sami moramo da odredimo kakvi ce biti i medjurezultati i sam izlaz
            y = Fxp(np.zeros(len(x)), signed = True, n_word = (x.n_word + int(max((a.n_word - a.n_frac), (b.n_word - b.n_frac)))+ int(max(a.n_frac, b.n_frac)) + int(max(len(a), len(b)))), n_frac = (x.n_frac + int(max(a.n_frac,b.n_frac))))
        
    
        
            y.overflow=x.overflow
            y.rounding=x.rounding
            
            
            leftLineCurrent=Fxp(np.zeros(len(b)), n_word = x.n_word + a.n_word + M , n_frac = x.n_frac + a.n_frac)  ##zbog for petlje smo stavili da currenta ima jedan vise
            leftLineNext=Fxp(np.zeros(len(b)-1), n_word = x.n_word + a.n_word + M, n_frac = x.n_frac + a.n_frac)
            rightLineCurrent=Fxp(np.zeros(len(b)), n_word = x.n_word + b.n_word + M ,  n_frac = x.n_frac + b.n_frac) ##zbog for petlje smo stavili da currenta ima jedan vise
            rightLineNext=Fxp(np.zeros(len(b)-1), n_word = x.n_word + b.n_word + M , n_frac = x.n_frac + b.n_frac)

    
            for n in range(N):
                
                
                leftLineCurrent[:len(b)-1]=leftLineNext
                rightLineCurrent[:len(bk)-1]=rightLineNext
                z=Fxp(leftLineCurrent[0]+x[n],like=y[0]/bk[0]-rightLineCurrent[0])
                y[n]=Fxp(bk[0]*z+rightLineCurrent[0],like=y)
                leftLineNext=leftLineCurrent[1:]-ak[1:]*z
                rightLineNext=rightLineCurrent[1:]+bk[1:]*z
                
   
        
        else:##ukoliko je zadatak format izlaznog broja 
        
        
            y = Fxp(np.zeros(len(x)), dtype = outFxpFormat)
            y.rounding=x.rounding
            y.overflow=x.overflow
            N=len(x)
            bk=Fxp(b,like=b)
            ak=Fxp(np.zeros(len(b)),like=a)
            ak[:len(a)]=a##sada imamo nizove ak i bk napravljeni tako da se dopuni ak nulama onoliko koliko je b niz veci od a 
            M=len(b)
    
    
            leftLineCurrent=Fxp(np.zeros(len(b)), like=y)  ##zbog for petlje smo stavili da currenta ima jedan vise
            leftLineNext=Fxp(np.zeros(len(b)-1), like=y)
            rightLineCurrent=Fxp(np.zeros(len(b)), like=y) ##zbog for petlje smo stavili da currenta ima jedan vise
            rightLineNext=Fxp(np.zeros(len(b)-1), like=y)
    
            for n in range(N):
                
                leftLineCurrent[:len(b)-1]=leftLineNext
                rightLineCurrent[:len(bk)-1]=rightLineNext
                z=leftLineCurrent[0]+x[n]
                y[n]=Fxp(bk[0]*z+rightLineCurrent[0], like=y)
                
                leftLineNext=leftLineCurrent[1:]-ak[1:]*z
                rightLineNext=rightLineCurrent[1:]+bk[1:]*z
        
               
        return y  
          

3.3

In [ ]:

#b=[1,2,3]
#a=[1,2,1]
fig,ax = plt.subplots(2,1,figsize = (10,5))
plt.subplots_adjust(bottom=0.25, wspace = 0.4)
# Definisanje ulaznog signala, zbir tri sinusoide
f1 = 100 
f2 = 270
f3 = 400
fs = 1000
N = 1000
n = np.arange(N) 
x = np.sin(2*np.pi*(f1/fs)*n) + 2*np.sin(2*np.pi*(f2/fs)*n)+3*np.sin(2*np.pi*(f3/fs)*n)

b,a = powerLineNoiseFilter(fs, f3, 40, 1)#
y = iirDirectTransposedAlternativeVersion(b, a, x)
# Provera ispravnosti funkcije
yScipy = signal.lfilter(b, a , x)#hocemo da poredimo sa ugradjenom funkcijom 
print("razlika rezultata ugradjene lfilter() i moje funkcije iirDirectTransposed je : {diff:.2f}.".format( diff=sum(abs(yScipy - y))))
#print(sum(abs(yScipy - y)))

# Stem input
ax[0].plot(n, x, label = 'Ulaz')
ax[0].plot(n, yScipy, label = 'Izlaz')
ax[0].set_xlabel(r'$n$')
ax[0].set_ylabel(r'$x[n], y[n]$')
ax[0].legend(loc = 'upper right');

f=fs*(np.arange(N)/N)
ax[1].plot(f[:N//2],np.abs(fft.fft(x))[:N//2],label='Ulaz')
ax[1].plot(f[:N//2],np.abs(fft.fft(y))[:N//2],label='Izlaz')
ax[1].set_xlabel(r'$f[Hz]$')#
ax[1].set_ylabel(r'$|X[n]|,|Y[n]|$')
ax[1].legend(loc = 'upper right');
 

3.4

In [ ]:
print(a)
print(b)
bFxp=Fxp(b,signed=True, n_word=18,n_frac=13,overflow='saturate',rounding='around')
aFxp=Fxp(a,signed=True, n_word=18,n_frac=13,overflow='saturate',rounding='around')
# vidimo da je najveca vrednos celog dela ovih koeficijenata 13 dakle treba nam najmanje 4 bita za ceo deo +znak jedan bit
m = np.roots(aFxp)
for p in range(len(m)):
        value = np.sqrt(m[p].real**2 + m[p].imag**2)
        
        if (value <= 1):
            print('stabilan')
            
        else:
            print('nestabilan')

print(aFxp)

fig = plt.figure(figsize = [6, 3])
plt.subplots_adjust(wspace = 0.3, hspace = 0.3)
ax = fig.add_subplot(1,1,1, aspect = 1)
zplane(bFxp, aFxp, ax)

ax.set_title('Raspored nula i polova(stabilan filter)')
W1, H1 = signal.freqz(b, a, worN = 1024)
W2, H2 = signal.freqz(bFxp, aFxp, worN = 1024)
f1=fs*(W1/(2*np.pi))
f2=fs*(W2/(2*np.pi))
fig, axs = plt.subplots(1, 1, figsize = [12,6], sharex=True)
plt.subplots_adjust(bottom=0.25, hspace = 0.7, left = 0.25)
axs.plot(f1,20*np.log10(abs(H1)), label = f'Originalna amplitudska karakteristika')
axs.plot(f2,20*np.log10(abs(H2)), 'r', label = f'Zaokružena amplitudska karakteristika')
axs.set_xlabel('$f[Hz]$')
axs.set_ylabel(r'$|H(e^{j\Omega}))| [dB]$')
axs.set_xlim([0, fs/2])
axs.set_ylim([-40, 5])
axs.set_title('Amplitudske karakteristike')
axs.legend(loc='best')

Ni pri jednoj promeni bitske sirine koeficijena a se ne desava da filter postane nestabilan

3.5

In [ ]:
print(max(x),min(x))

In [ ]:
xFxp=Fxp(x, signed = True, n_word = 14, n_frac = 10, overflow = 'saturate', rounding = 'around')

y = iirDirectTransposedAlternativeVersion(b, a, x)
print(max(y))
yFxp = iirDirectTransposedFXP(bFxp, aFxp, xFxp,outFxpFormat='fxp-s20/10')
fig, axs = plt.subplots(4, 1, figsize = [12, 8])
             
axs[0].plot(n,x)
axs[0].set_ylabel('$x[n]$')

axs[1].plot(n,y)
axs[1].set_ylabel('$y[n]$')


axs[2].plot(n,yFxp)
axs[2].set_ylabel('$yFxp[n]$')

z=y-yFxp
axs[3].plot(n,z)
axs[3].set_ylabel('$y-yFxp$')
axs[3].set_xlabel('$n$')



4.1

In [ ]:
 def iirDirectTransposedCascade(b, a, x):
        numOFfilters = len(b)
        stepResults = np.zeros((numOFfilters+1, len(x)))
        stepResults[0] = x
        for i in range(1,numOFfilters+1):
            stepResults[i] = iirDirectTransposedAlternativeVersion(b[i-1], a[i-1], stepResults[i-1]) 
            
        y = stepResults[numOFfilters]
        return y 

4.3

In [ ]:
fig,ax = plt.subplots(2,1,figsize = (10,5))
plt.subplots_adjust(bottom=0.25, wspace = 0.4)
# Definisanje ulaznog signala, zbir tri sinusoide
f1 = 100 
f2 = 270
f3 = 400
fs = 1000
N = 1000
n = np.arange(N) 
x = np.sin(2*np.pi*(f1/fs)*n) + 2*np.sin(2*np.pi*(f2/fs)*n)+3*np.sin(2*np.pi*(f3/fs)*n)

b1,a1 = powerLineNoiseFilter(fs, f1, 40, 1)
b2,a2 = powerLineNoiseFilter(fs, f2, 40, 1)
b3,a3 = powerLineNoiseFilter(fs, f3, 40, 1)
#print(b,a)
y = iirDirectTransposedCascade([b1,b2],[a1,a2], x)
#y = iirDirectTransposedCascade([b1,b2,b3],[a1,a2,a3], x)

ax[0].plot(n, x, label = 'Ulazni signal')
ax[0].plot(n, y, label = 'Izlaz signal')
ax[0].set_xlabel(r'$n$')
ax[0].set_ylabel(r'$x[n], y[n]$')
ax[0].legend(loc = 'upper right');

f=fs*(np.arange(N)/N)
ax[1].plot(f[:N//2],np.abs(fft.fft(x))[:N//2],label='Ulazni signal')
ax[1].plot(f[:N//2],np.abs(fft.fft(y))[:N//2],label='Izlazni signal')
ax[1].set_xlabel(r'$f[Hz]$')#
ax[1].set_ylabel(r'$|X[n]|,|Y[n]|$')
ax[1].legend(loc = 'upper right');

In [ ]:
pip install fxpmath
